In [2]:
import import_ipynb
# para recarregar do modelo quando tiver alguma alteração
import importlib
#importlib.reload(U)

import os
import numpy as np
import tensorflow as tf
from keras.models import load_model
from matplotlib import pyplot as plt

import time
from termcolor import colored

import utils as U

importing Jupyter notebook from utils.ipynb


In [12]:
def read_image(image_path, patch_size):
    image = U.load_image(image_path)
    image_data = U.convert_to_array(image)
    image_data[np.isnan(image_data)] = 0 # adiciona 0 onde é NaN
    image_data = U.normalize_(image_data)

    """mask = U.load_image(mask_path)
    mask_data = U.convert_to_array(mask)
    mask_data[np.isnan(mask_data)] = 0 # adiciona 0 onde é NaN
    mask_data[mask_data>0] = 1
    mask_data[mask_data<=0] = 0
    """

    all_images = []
    ps = patch_size
    for i in range(0, image_data.shape[1], ps):
        for j in range(0, image_data.shape[1], ps):
            img = image_data[i:i+ps,j:j+ps,:]
            #img = img[:,:,[0,1,2,6,7,9,13,14]]
            all_images.append(img)

    images = np.array(all_images)
    
    return image, image_data, images#, mask_data

In [8]:
model = load_model("./Models/deforestation_trained_model_samples_mapbiomas_unet2_3.h5", custom_objects={'soft_dice_loss': U.soft_dice_loss, 'iou_coef': U.iou_coef})

In [11]:
def render_prediction(images, patch_size):
    predictions = model.predict(images, verbose=0)
    predictions = predictions.squeeze()

    predicted = np.zeros((256, 256))
    
    ps = patch_size
    x = 0
    for i in range(0, 256, ps):
        for j in range(0, 256, ps):
            predicted[i:i+ps, j:j+ps] += predictions[x]
            x += 1
    
    predicted = np.where(predicted > 0.5, 1, 0)
    
    return predicted

In [5]:
def show_predictions(image_data, mask_data, predicted):

    f = plt.figure(figsize=(20,20))

    f.add_subplot(3, 3, 1)
    # 2 = B4, 1 = B3, 0 = B2
    plt.imshow(tf.keras.preprocessing.image.array_to_img(image_data[:,:,[2,1,0]]))
    plt.title('Image')

    f.add_subplot(3, 3, 2)
    plt.imshow(mask_data)
    plt.title('Truth mask')

    f.add_subplot(3, 3, 3)
    plt.imshow(predicted)
    plt.title('Prediction')

    plt.show()

In [9]:
def merge_images(predicted_images):
    output = 'Data/Predicted/test_samples_mapbiomas_unet2_3_test2.tif'
    os.system('gdal_merge.py -n 0 -co COMPRESS=PACKBITS -co BIGTIFF=YES -of gtiff {0} -o {1}'.format(predicted_images, output))

In [10]:
images_path = 'Data/BuildingsDataSet/Prediction/Images/'
#masks_path = 'Data/BuildingsDataSet/Prediction/Masks/'

patch_size = 64
predicted_images = []
file_out_n = 'Data/Predicted/teste'

start_time = time.time()

files = next(os.walk(images_path))[2]
for i, image_name in enumerate(files):

    image_path = images_path + image_name
    #mask_path = masks_path + image_name
    
    image, image_data, images = read_image(image_path, patch_size)
    predicted = render_prediction(images, patch_size)
    #show_predictions(image_data, mask_data, predicted)
    
    output = '{0}{1}.tif'.format(file_out_n, i)
    U.convert_to_raster(image, predicted, output)
    
    predicted_images.append(output)
    
predicted_images = " ".join(predicted_images)

merge_images(predicted_images)

# após fazer o merge deleta as imagens
os.system('rm -rf {0}*'.format(file_out_n))

# spent time
elapsed_time = time.time() - start_time
print(colored('Spent time: {0}'.format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))), 'red'))
 

Spent time: 00:01:18
